In [1]:
import logging
import imp
from dateutil.relativedelta import relativedelta

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [3]:
from helpers.dataset import read_quote_dataset, preprocess_quotes
from helpers.backtest import train_model_and_backtest_regressor, get_backtest_performance_metrics

In [4]:
# Configir logging module for jypter notebook
imp.reload(logging)
logging_format = '%(asctime)s - %(levelname)s - %(process)s - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=logging_format)

In [5]:
PARAM_DATASET = '../data/SPY_postprocess_adj.csv.gz'

In [6]:
df = read_quote_dataset(PARAM_DATASET)

In [7]:
df.head()

,date,open,high,low,close,close_adj,volume,open_adj,low_adj,high_adj,...,ratio_close_adj_000_close_adj_005_norm,ratio_close_adj_000_close_adj_020_norm,ratio_close_adj_000_ema_005_norm,ratio_close_adj_000_ema_010_norm,ratio_close_adj_000_ema_020_norm,ratio_close_adj_000_ema_050_norm,ratio_close_adj_000_sma_005_norm,ratio_close_adj_000_sma_010_norm,ratio_close_adj_000_sma_020_norm,ratio_close_adj_000_sma_050_norm
0,2000-01-03,148.250000,148.250000,143.875000,145.4375,101.425385,8164300,103.386770,100.335727,103.386770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,143.531204,144.062500,139.640594,139.7500,97.459068,8089800,100.096010,97.382770,100.466526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,139.937500,141.531204,137.250000,140.0000,97.633377,12177900,97.589791,95.715579,98.701210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,139.625000,141.500000,137.750000,137.7500,96.064301,6227200,97.371891,96.064301,98.679482,...,NaN,NaN,0.486630,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,140.312500,145.750000,140.062500,145.7500,101.643333,8066500,97.851322,97.676977,101.643333,...,NaN,NaN,0.815422,NaN,NaN,NaN,0.740588,NaN,NaN,NaN


In [8]:
vars_to_shift = ['close_adj', 'close_adj_norm', 'close_adj_std']
shift_periods = [1, 5, 10, 20]
vars_for_return = ['close_adj']
return_periods = [1, 5, 10, 20]

In [9]:
df = preprocess_quotes(df, vars_to_shift=vars_to_shift, shift_periods=shift_periods,
                       vars_for_return=vars_for_return, return_periods=return_periods)

In [10]:
df[['date', 'close_adj', 'close_adj_shift_1', 'close_adj_ret_1', 'close_adj_shift_5', 'close_adj_ret_5']].head(10)

,date,close_adj,close_adj_shift_1,close_adj_ret_1,close_adj_shift_5,close_adj_ret_5
0,2000-01-03,101.425385,97.459068,-0.039106,101.992004,0.005587
1,2000-01-04,97.459068,97.633377,0.001789,100.771645,0.033989
2,2000-01-05,97.633377,96.064301,-0.016071,99.769150,0.021875
3,2000-01-06,96.064301,101.643333,0.058076,101.120308,0.052631
4,2000-01-07,101.643333,101.992004,0.003430,102.493233,0.008362
5,2000-01-10,101.992004,100.771645,-0.011965,101.686958,-0.002991
6,2000-01-11,100.771645,99.769150,-0.009948,102.515060,0.017301
7,2000-01-12,99.769150,101.120308,0.013543,100.945953,0.011795
8,2000-01-13,101.120308,102.493233,0.013577,100.727989,-0.003880
9,2000-01-14,102.493233,101.686958,-0.007867,97.873047,-0.045078


In [11]:
x_vars = ['close_adj']
y_var = 'close_adj_shift_1'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

In [12]:
df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1),
    ignore_last_x_training_items=0)

2019-05-05 16:10:14,862 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:15,179 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:15,189 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:15,191 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:15,200 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:15,203 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:15,210 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:15,213 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:15,225 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:15,227 - INFO - 22848 - Training dataset is between 2002-04-30 and 2002-05-30.
2019-05-05 16:10:15,237 - INFO - 22848 - Training a model to be tested between 2002-06-30 and 20

2019-05-05 16:10:15,666 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:15,675 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:15,678 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:15,687 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:15,689 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:15,698 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:15,700 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:15,709 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:15,712 - INFO - 22848 - Training dataset is between 2005-09-30 and 2005-10-28.
2019-05-05 16:10:15,720 - INFO - 22848 - Training a model to be tested between 2005-11-30 and 20

2019-05-05 16:10:16,144 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:16,156 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:16,159 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:16,170 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:16,172 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:16,181 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:16,184 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:16,193 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:16,194 - INFO - 22848 - Training dataset is between 2009-03-02 and 2009-03-30.
2019-05-05 16:10:16,204 - INFO - 22848 - Training a model to be tested between 2009-04-30 and 20

2019-05-05 16:10:16,598 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:16,606 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:16,609 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:16,618 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:16,620 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:16,628 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:16,630 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:16,639 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:16,641 - INFO - 22848 - Training dataset is between 2012-07-31 and 2012-08-30.
2019-05-05 16:10:16,649 - INFO - 22848 - Training a model to be tested between 2012-09-30 and 20

2019-05-05 16:10:17,034 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:17,043 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:17,045 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:17,053 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:17,055 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:17,064 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:17,066 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:17,074 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:17,076 - INFO - 22848 - Training dataset is between 2015-12-31 and 2016-01-29.
2019-05-05 16:10:17,085 - INFO - 22848 - Training a model to be tested between 2016-02-29 and 20

In [13]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.017836,-3.322314e-16,NaN
beta,0.290970,1.000000e+00,NaN
cagr,0.019330,5.047234e-02,-0.031143
max_drawdown,-0.494455,-5.518942e-01,0.057439
return,0.435562,1.534235e+00,-1.098673
sharpe,0.195584,3.527034e-01,-0.157119
var,-0.017761,-1.925716e-02,0.001496
volatility,0.191751,1.916436e-01,0.000107


In [14]:
x_vars = ['open_adj', 'low_adj', 'high_adj', 'close_adj']
y_var = 'close_adj_shift_1'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1))

2019-05-05 16:10:17,828 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:18,141 - INFO - 22848 - Training a model to be tested between 2002-01-31 and 2002-02-28.
2019-05-05 16:10:18,144 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:18,156 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:18,158 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:18,168 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:18,170 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:18,180 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:18,182 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:18,195 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:18,197 - INFO - 22848 - Training dataset is between 2002-04-30 and 20

2019-05-05 16:10:18,602 - INFO - 22848 - Training a model to be tested between 2005-06-30 and 2005-07-31.
2019-05-05 16:10:18,604 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:18,613 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:18,617 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:18,628 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:18,630 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:18,639 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:18,641 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:18,650 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:18,653 - INFO - 22848 - Training dataset is between 2005-09-30 and 20

2019-05-05 16:10:19,077 - INFO - 22848 - Training a model to be tested between 2008-11-30 and 2008-12-31.
2019-05-05 16:10:19,080 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:19,088 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:19,090 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:19,099 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:19,101 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:19,110 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:19,112 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:19,119 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:19,122 - INFO - 22848 - Training dataset is between 2009-03-02 and 20

2019-05-05 16:10:19,514 - INFO - 22848 - Training a model to be tested between 2012-04-30 and 2012-05-31.
2019-05-05 16:10:19,515 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:19,524 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:19,526 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:19,535 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:19,537 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:19,545 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:19,547 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:19,556 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:19,558 - INFO - 22848 - Training dataset is between 2012-07-31 and 20

2019-05-05 16:10:19,939 - INFO - 22848 - Training a model to be tested between 2015-09-30 and 2015-10-31.
2019-05-05 16:10:19,942 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:19,950 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:19,951 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:19,961 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:19,964 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:19,973 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:19,976 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:19,985 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:19,987 - INFO - 22848 - Training dataset is between 2015-12-31 and 20

In [15]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.062292,-3.322314e-16,NaN
beta,0.183535,1.000000e+00,NaN
cagr,0.057940,5.047234e-02,0.007468
max_drawdown,-0.313108,-5.518942e-01,0.238787
return,1.896996,1.534235e+00,0.362761
sharpe,0.389161,3.527034e-01,0.036458
var,-0.017778,-1.925716e-02,0.001479
volatility,0.191945,1.916436e-01,0.000301


In [16]:
x_vars = ['low_adj', 'high_adj']
y_var = 'close_adj_shift_1'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1))

2019-05-05 16:10:22,714 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:23,007 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:23,016 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:23,019 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:23,028 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:23,030 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:23,042 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:23,044 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:23,055 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:23,057 - INFO - 22848 - Training dataset is between 2002-04-30 and 2002-05-30.
2019-05-05 16:10:23,067 - INFO - 22848 - Training a model to be tested between 2002-06-30 and 20

2019-05-05 16:10:23,478 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:23,490 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:23,492 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:23,503 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:23,505 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:23,515 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:23,517 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:23,525 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:23,527 - INFO - 22848 - Training dataset is between 2005-09-30 and 2005-10-28.
2019-05-05 16:10:23,536 - INFO - 22848 - Training a model to be tested between 2005-11-30 and 20

2019-05-05 16:10:23,921 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:23,931 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:23,933 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:23,941 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:23,943 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:23,951 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:23,953 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:23,962 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:23,964 - INFO - 22848 - Training dataset is between 2009-03-02 and 2009-03-30.
2019-05-05 16:10:23,973 - INFO - 22848 - Training a model to be tested between 2009-04-30 and 20

2019-05-05 16:10:24,349 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:24,356 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:24,358 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:24,365 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:24,368 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:24,377 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:24,379 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:24,386 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:24,388 - INFO - 22848 - Training dataset is between 2012-07-31 and 2012-08-30.
2019-05-05 16:10:24,398 - INFO - 22848 - Training a model to be tested between 2012-09-30 and 20

2019-05-05 16:10:24,771 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:24,780 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:24,782 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:24,790 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:24,792 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:24,803 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:24,805 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:24,814 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:24,816 - INFO - 22848 - Training dataset is between 2015-12-31 and 2016-01-29.
2019-05-05 16:10:24,825 - INFO - 22848 - Training a model to be tested between 2016-02-29 and 20

In [17]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.096132,-3.322314e-16,NaN
beta,0.209123,1.000000e+00,NaN
cagr,0.096241,5.047234e-02,0.045769
max_drawdown,-0.317199,-5.518942e-01,0.234695
return,4.670535,1.534235e+00,3.136300
sharpe,0.574377,3.527034e-01,0.221674
var,-0.017437,-1.925716e-02,0.001820
volatility,0.191977,1.916436e-01,0.000333


In [18]:
x_vars = ['open_adj', 'low_adj', 'high_adj']
y_var = 'close_adj_shift_1'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1))

2019-05-05 16:10:25,954 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:26,247 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:26,257 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:26,259 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:26,270 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:26,273 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:26,283 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:26,285 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:26,295 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:26,297 - INFO - 22848 - Training dataset is between 2002-04-30 and 2002-05-30.
2019-05-05 16:10:26,308 - INFO - 22848 - Training a model to be tested between 2002-06-30 and 20

2019-05-05 16:10:26,701 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:26,710 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:26,712 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:26,720 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:26,723 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:26,732 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:26,734 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:26,742 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:26,744 - INFO - 22848 - Training dataset is between 2005-09-30 and 2005-10-28.
2019-05-05 16:10:26,752 - INFO - 22848 - Training a model to be tested between 2005-11-30 and 20

2019-05-05 16:10:27,155 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:27,163 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:27,165 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:27,173 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:27,176 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:27,184 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:27,186 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:27,193 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:27,196 - INFO - 22848 - Training dataset is between 2009-03-02 and 2009-03-30.
2019-05-05 16:10:27,205 - INFO - 22848 - Training a model to be tested between 2009-04-30 and 20

2019-05-05 16:10:27,591 - INFO - 22848 - Training a model to be tested between 2012-04-30 and 2012-05-31.
2019-05-05 16:10:27,593 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:27,601 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:27,603 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:27,612 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:27,614 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:27,623 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:27,625 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:27,633 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:27,635 - INFO - 22848 - Training dataset is between 2012-07-31 and 20

2019-05-05 16:10:28,057 - INFO - 22848 - Training a model to be tested between 2015-09-30 and 2015-10-31.
2019-05-05 16:10:28,060 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:28,069 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:28,070 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:28,079 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:28,082 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:28,090 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:28,092 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:28,101 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:28,103 - INFO - 22848 - Training dataset is between 2015-12-31 and 20

In [19]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.125131,-3.322314e-16,NaN
beta,0.153543,1.000000e+00,NaN
cagr,0.124259,5.047234e-02,0.073787
max_drawdown,-0.313869,-5.518942e-01,0.238025
return,8.132907,1.534235e+00,6.598672
sharpe,0.705790,3.527034e-01,0.353087
var,-0.017454,-1.925716e-02,0.001804
volatility,0.191997,1.916436e-01,0.000353


In [20]:
x_vars = ['open_adj', 'low_adj', 'high_adj', 'close_adj']
y_var = 'close_adj_shift_1'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1))

2019-05-05 16:10:28,696 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:28,989 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:28,999 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:29,001 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:29,011 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:29,013 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:29,022 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:29,025 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:29,035 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:29,038 - INFO - 22848 - Training dataset is between 2002-04-30 and 2002-05-30.
2019-05-05 16:10:29,048 - INFO - 22848 - Training a model to be tested between 2002-06-30 and 20

2019-05-05 16:10:29,452 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:29,460 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:29,461 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:29,470 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:29,472 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:29,479 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:29,483 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:29,492 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:29,494 - INFO - 22848 - Training dataset is between 2005-09-30 and 2005-10-28.
2019-05-05 16:10:29,503 - INFO - 22848 - Training a model to be tested between 2005-11-30 and 20

2019-05-05 16:10:29,910 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:29,918 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:29,920 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:29,929 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:29,931 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:29,940 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:29,942 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:29,954 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:29,957 - INFO - 22848 - Training dataset is between 2009-03-02 and 2009-03-30.
2019-05-05 16:10:29,968 - INFO - 22848 - Training a model to be tested between 2009-04-30 and 20

2019-05-05 16:10:30,349 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:30,360 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:30,363 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:30,373 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:30,376 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:30,385 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:30,388 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:30,396 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:30,398 - INFO - 22848 - Training dataset is between 2012-07-31 and 2012-08-30.
2019-05-05 16:10:30,406 - INFO - 22848 - Training a model to be tested between 2012-09-30 and 20

2019-05-05 16:10:30,813 - INFO - 22848 - Training a model to be tested between 2015-09-30 and 2015-10-31.
2019-05-05 16:10:30,817 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:30,829 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:30,831 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:30,840 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:30,843 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:30,854 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:30,859 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:30,877 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:30,881 - INFO - 22848 - Training dataset is between 2015-12-31 and 20

In [21]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.062292,-3.322314e-16,NaN
beta,0.183535,1.000000e+00,NaN
cagr,0.057940,5.047234e-02,0.007468
max_drawdown,-0.313108,-5.518942e-01,0.238787
return,1.896996,1.534235e+00,0.362761
sharpe,0.389161,3.527034e-01,0.036458
var,-0.017778,-1.925716e-02,0.001479
volatility,0.191945,1.916436e-01,0.000301


In [22]:
x_vars = ['open_adj', 'low_adj', 'high_adj']
y_var = 'close_adj_shift_20'
buy_price_col = 'close_adj'
sell_price_col = 'close_adj_shift_1'
model_class = LinearRegression
model_params = {'fit_intercept': True}

df_backtest = train_model_and_backtest_regressor(df, x_vars=x_vars, y_var=y_var, 
    buy_price_col=buy_price_col, sell_price_col=sell_price_col,
    model_class=model_class, model_params=model_params, 
    backtest_start='2000-02-01', backtest_end='2018-12-31', 
    model_update_frequency='M', train_history_period=relativedelta(months=1))

2019-05-05 16:10:31,431 - DEBUG - 22848 - 228 periods to backtest: ['2000-02-01', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07

2019-05-05 16:10:31,748 - INFO - 22848 - Training dataset is between 2001-12-31 and 2002-01-30.
2019-05-05 16:10:31,762 - INFO - 22848 - Training a model to be tested between 2002-02-28 and 2002-03-31.
2019-05-05 16:10:31,766 - INFO - 22848 - Training dataset is between 2002-01-28 and 2002-02-27.
2019-05-05 16:10:31,777 - INFO - 22848 - Training a model to be tested between 2002-03-31 and 2002-04-30.
2019-05-05 16:10:31,781 - INFO - 22848 - Training dataset is between 2002-02-28 and 2002-03-28.
2019-05-05 16:10:31,792 - INFO - 22848 - Training a model to be tested between 2002-04-30 and 2002-05-31.
2019-05-05 16:10:31,794 - INFO - 22848 - Training dataset is between 2002-04-01 and 2002-04-29.
2019-05-05 16:10:31,806 - INFO - 22848 - Training a model to be tested between 2002-05-31 and 2002-06-30.
2019-05-05 16:10:31,808 - INFO - 22848 - Training dataset is between 2002-04-30 and 2002-05-30.
2019-05-05 16:10:31,820 - INFO - 22848 - Training a model to be tested between 2002-06-30 and 20

2019-05-05 16:10:32,251 - INFO - 22848 - Training a model to be tested between 2005-06-30 and 2005-07-31.
2019-05-05 16:10:32,254 - INFO - 22848 - Training dataset is between 2005-05-31 and 2005-06-29.
2019-05-05 16:10:32,263 - INFO - 22848 - Training a model to be tested between 2005-07-31 and 2005-08-31.
2019-05-05 16:10:32,266 - INFO - 22848 - Training dataset is between 2005-06-30 and 2005-07-29.
2019-05-05 16:10:32,274 - INFO - 22848 - Training a model to be tested between 2005-08-31 and 2005-09-30.
2019-05-05 16:10:32,276 - INFO - 22848 - Training dataset is between 2005-08-01 and 2005-08-30.
2019-05-05 16:10:32,285 - INFO - 22848 - Training a model to be tested between 2005-09-30 and 2005-10-31.
2019-05-05 16:10:32,287 - INFO - 22848 - Training dataset is between 2005-08-30 and 2005-09-29.
2019-05-05 16:10:32,296 - INFO - 22848 - Training a model to be tested between 2005-10-31 and 2005-11-30.
2019-05-05 16:10:32,298 - INFO - 22848 - Training dataset is between 2005-09-30 and 20

2019-05-05 16:10:32,704 - INFO - 22848 - Training a model to be tested between 2008-11-30 and 2008-12-31.
2019-05-05 16:10:32,707 - INFO - 22848 - Training dataset is between 2008-10-30 and 2008-11-28.
2019-05-05 16:10:32,717 - INFO - 22848 - Training a model to be tested between 2008-12-31 and 2009-01-31.
2019-05-05 16:10:32,719 - INFO - 22848 - Training dataset is between 2008-12-01 and 2008-12-30.
2019-05-05 16:10:32,728 - INFO - 22848 - Training a model to be tested between 2009-01-31 and 2009-02-28.
2019-05-05 16:10:32,730 - INFO - 22848 - Training dataset is between 2008-12-31 and 2009-01-30.
2019-05-05 16:10:32,740 - INFO - 22848 - Training a model to be tested between 2009-02-28 and 2009-03-31.
2019-05-05 16:10:32,741 - INFO - 22848 - Training dataset is between 2009-01-28 and 2009-02-27.
2019-05-05 16:10:32,750 - INFO - 22848 - Training a model to be tested between 2009-03-31 and 2009-04-30.
2019-05-05 16:10:32,752 - INFO - 22848 - Training dataset is between 2009-03-02 and 20

2019-05-05 16:10:33,208 - INFO - 22848 - Training a model to be tested between 2012-04-30 and 2012-05-31.
2019-05-05 16:10:33,210 - INFO - 22848 - Training dataset is between 2012-03-30 and 2012-04-27.
2019-05-05 16:10:33,222 - INFO - 22848 - Training a model to be tested between 2012-05-31 and 2012-06-30.
2019-05-05 16:10:33,224 - INFO - 22848 - Training dataset is between 2012-04-30 and 2012-05-30.
2019-05-05 16:10:33,233 - INFO - 22848 - Training a model to be tested between 2012-06-30 and 2012-07-31.
2019-05-05 16:10:33,236 - INFO - 22848 - Training dataset is between 2012-05-30 and 2012-06-29.
2019-05-05 16:10:33,248 - INFO - 22848 - Training a model to be tested between 2012-07-31 and 2012-08-31.
2019-05-05 16:10:33,251 - INFO - 22848 - Training dataset is between 2012-07-02 and 2012-07-30.
2019-05-05 16:10:33,259 - INFO - 22848 - Training a model to be tested between 2012-08-31 and 2012-09-30.
2019-05-05 16:10:33,262 - INFO - 22848 - Training dataset is between 2012-07-31 and 20

2019-05-05 16:10:33,653 - INFO - 22848 - Training a model to be tested between 2015-09-30 and 2015-10-31.
2019-05-05 16:10:33,655 - INFO - 22848 - Training dataset is between 2015-08-31 and 2015-09-29.
2019-05-05 16:10:33,663 - INFO - 22848 - Training a model to be tested between 2015-10-31 and 2015-11-30.
2019-05-05 16:10:33,665 - INFO - 22848 - Training dataset is between 2015-09-30 and 2015-10-30.
2019-05-05 16:10:33,674 - INFO - 22848 - Training a model to be tested between 2015-11-30 and 2015-12-31.
2019-05-05 16:10:33,676 - INFO - 22848 - Training dataset is between 2015-10-30 and 2015-11-27.
2019-05-05 16:10:33,684 - INFO - 22848 - Training a model to be tested between 2015-12-31 and 2016-01-31.
2019-05-05 16:10:33,686 - INFO - 22848 - Training dataset is between 2015-11-30 and 2015-12-30.
2019-05-05 16:10:33,694 - INFO - 22848 - Training a model to be tested between 2016-01-31 and 2016-02-29.
2019-05-05 16:10:33,696 - INFO - 22848 - Training dataset is between 2015-12-31 and 20

In [23]:
get_backtest_performance_metrics(df_backtest.ret, df_backtest.benchmark_ret, with_benchmark=True, with_delta=True)

,main,benchmark,delta
alpha,0.549535,-3.322314e-16,NaN
beta,-0.055703,1.000000e+00,NaN
cagr,0.694489,5.047234e-02,0.644017
max_drawdown,-0.194069,-5.518942e-01,0.357825
return,21152.394548,1.534235e+00,21150.860313
sharpe,2.885469,3.527034e-01,2.532765
var,-0.014543,-1.925716e-02,0.004714
volatility,0.189144,1.916436e-01,-0.002499


Model with 69% return yearly, we are millonaires. Here there is look ahead bias.